In [1]:
import torch
import numpy as np
from datasets import load_dataset
from string import punctuation
from collections import Counter

In [2]:
#!pip install --upgrade datasets
dataset = load_dataset("cnn_dailymail","3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
len(dataset['train'])

287113

In [4]:
articles = np.array(dataset['train'][:10000]['article'])
summaries = np.array(dataset['train'][:10000]['highlights'])
text = {'articles':articles,
        'summaries':summaries}

#Pre-Processing

In [5]:
def removing_unwanted_characters(text):
    punctuations = punctuation.replace("'","")
    text = "".join([c for c in text if c not in punctuations])
    text = text.split("\n")
    text = " ".join(text)
    text = text.strip()
    text = " ".join(text.split())

    return text

In [6]:
def tokenizing(data):
    all_unique_words = []
    for text in data:
        all_unique_words.extend(text.split())
    vocab = set(all_unique_words)
    vocab = {word:ii for ii, word in enumerate(vocab, 1)}
    return vocab

In [7]:
def preprocessing(data):
    data = np.array([text.lower() for text in data])
    items = data.shape[0]
    for i in range(items):
       data[i] = removing_unwanted_characters(data[i])

    vocab = tokenizing(data)

    data_ints = []
    for text in data:
        data_ints.append([vocab.get(word) for word in text.split() if vocab.get(word) != None])

    return data_ints, vocab

In [8]:
data, data_tokenz = preprocessing(text['articles'])
target, target_tokenz = preprocessing(text['summaries'])

In [ ]:
data_lengths = Counter([len(x) for x in data])
target_lengths = Counter([len(x) for x in target])

In [ ]:
sorted(list(data_lengths.keys()))[:5]

[14, 20, 36, 42, 46]

In [ ]:
len(max(data))

194

In [ ]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[1])

Unique words:  106986

Tokenized review: 
 [2283, 1104, 6, 85, 434, 1, 2583, 805, 39, 12061, 774, 40, 2726, 6, 3550, 178, 5, 9162, 1, 1134, 434, 1, 958, 208, 13266, 5324, 957, 1888, 567, 4, 1208, 89, 95, 3, 1, 2968, 22, 4798, 2476, 26, 5920, 5699, 12, 1, 5182, 1480, 89, 95, 4798, 2476, 2968, 22, 5699, 6, 1669, 94, 660, 1669, 465, 39, 1, 6224, 1480, 3, 1, 11839, 13975, 2491, 1235, 10, 3767, 1, 5182, 1480, 208, 2968, 14, 1, 97, 1711, 1288, 6503, 22, 10746, 258, 359, 1183, 2, 1363, 6, 172, 97, 445, 30, 373, 417, 401, 45, 401, 3, 7630, 26, 566, 401, 7, 516, 3254, 22290, 79, 22, 1382, 23502, 16708, 13, 79, 1, 2184, 445, 765, 21, 13976, 14, 78, 4798, 2476, 43, 445, 990, 77, 55, 359, 67, 47, 78, 2834, 12, 1, 714, 6370, 1101, 2, 22291, 40, 2205, 5, 30, 365, 46, 17276, 28586, 5, 413, 611, 2, 1560, 6950, 88, 2, 22290, 61, 30, 249, 57, 12, 1, 6224, 1480, 4704, 4798, 8066, 24, 27, 419, 125, 410, 162, 81, 359, 6, 1208, 35, 9457, 1, 1208, 14, 22290, 13, 10, 167, 289, 6, 1669, 16, 26, 4507, 9, 497, 5